# Exhaustive search - Flowshop problem

### About this notebook
This notebook contains a hands-on the flowshop problem. We will focus on implementing the exact methods to solve this problem. These methods guarantee to find the optimal solution for our problem. Note that some heuristics such as NEH are also being implemented in order to initialize the best makespan..

### Used ressources
- [Finding an Optimal Sequence in the Flowshop Scheduling Using Johnson’s Algorithm](https://ijiset.com/vol2/v2s1/IJISET_V2_I1_50.pdf)
- [Benchmarks for Basic Scheduling Problems](http://mistic.heig-vd.ch/taillard/articles.dir/Taillard1993EJOR.pdf)

In [1]:
import numpy as np
import matplotlib as plt
import itertools
import time

# Brute force

In [2]:
def all_permutations(iterable):
    permutations = list(itertools.permutations(iterable))
    permutations_as_lists = [list(p) for p in permutations]
    return permutations_as_lists

In [3]:
'''
    This function calculates the makespan of a given sequence.
    The makespan is the time it takes to complete all the jobs on all the machines.
    
    Inputs:
        processing_times : the matrix that contains the execution time of each Job in each machine.
        sequence :  a sequence of jobs e.g [0, 2, 1].
    
    Output: The makespan of the sequence.
'''
def evaluate_sequence(sequence, processing_times):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)
    completion_times = np.zeros((num_jobs, num_machines))
    
    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]
    
    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]
    
    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]

In [4]:
'''
    This function will return the sequence with the smallest makespan.

    Inputs:
        processing_times : the matrix that contains the execution time of each Job in each machine.
        permutations : all possible sequences.
    
    Outputs:
        sol : the optimal sequence.
        M : the makespan of the sequence
'''

def brute_force(processing_times, permutations):
    M = float('inf')
    sol = []
    for permutation in permutations:
        m = evaluate_sequence(permutation, processing_times)
        if m < M:
            M = m
            sol = permutation
    return sol, M

# NEH Algorithm

In [15]:
def order_jobs_in_descending_order_of_total_completion_time(processing_times):
    total_completion_time = processing_times.sum(axis=1)
    return np.argsort(total_completion_time, axis=0).tolist()

In [16]:
def insertion(sequence, position, value):
    new_seq = sequence[:]
    new_seq.insert(position, value)
    return new_seq

In [17]:
def neh_algorithm(processing_times):
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)
    # Define the initial order
    J1, J2 = ordered_sequence[:2]
    sequence = [J1, J2] if evaluate_sequence([J1, J2], processing_times) < evaluate_sequence([J2, J1], processing_times) else [J2, J1]
    del ordered_sequence[:2]
    # Add remaining jobs
    for job in ordered_sequence:
        Cmax = float('inf')
        best_sequence = []
        for i in range(len(sequence)+1):
            new_sequence = insertion(sequence, i, job)
            Cmax_eval = evaluate_sequence(new_sequence, processing_times)
            if Cmax_eval < Cmax:
                Cmax = Cmax_eval
                best_sequence = new_sequence
        sequence = best_sequence
    return sequence, Cmax

# Johnson Algorithm

### Description
Johnson's algorithm is a way to find the shortest paths between all pairs of vertices in a sparse directed graph. In
operations research Johnson's rule is a method of scheduling jobs in two work centers. Its primary objective is to find an optimal sequence of jobs to reduce makespan (the total amount of time it takes to complete all jobs). It also reduces the number of idle time between the two work centers. Results are not always optimal, especially for a small group of jobs.
### Idea behind 
- List the jobs and their times at each work center.
- Select the job with the shortest activity time. If that activity time is for the first work center, then schedule the job first. If that activity time is for the second work center then schedule the job last. Break ties arbitrarily.
- Eliminate the shortest job from further consideration.

In [5]:
'''
    The johnson method is only possible when we have two machines.
    
    Input : the matrix that contains the execution time of each Job in each machine.
    Output : the optimal sequence.
'''

def johnson_method(processing_times):
    jobs, machines = processing_times.shape
    copy_processing_times = processing_times.copy()
    maximum = processing_times.max() + 1
    m1 = []
    m2 = []
    
    if machines != 2:
        raise Exception("Johson method only works with two machines")
        
    for i in range(jobs):
        minimum = copy_processing_times.min()
        position = np.where(copy_processing_times == minimum)
        
        if position[1][0] == 0:
            m1.append(position[0][0])
        else:
            m2.insert(0, position[0][0])
        
        copy_processing_times[position[0][0]] = maximum
        # Delete the job appended
        
    return m1+m2

### Tests

In [7]:
# Generate a random example to work with 7 jobs and 2 machines
rnd_data = np.random.randint(size=(7,2), low=5, high=23)
print(rnd_data, "\n")

start_time = time.time()
sol = johnson_method(rnd_data)
elapsed_time = time.time() - start_time

print(f'Best sequence found by Johnson is {sol} with a makespan of {evaluate_sequence(sol, rnd_data)}')
print("Elapsed time:", elapsed_time, "seconds")

[[22 17]
 [21 16]
 [ 6 22]
 [ 6 20]
 [15 19]
 [15 17]
 [21 20]] 

Best sequence found by Johnson is [2, 3, 4, 5, 6, 0, 1] with a makespan of 137.0
Elapsed time: 0.0012052059173583984 seconds


In [8]:
init_jobs = 7
init_job_list = list(range(init_jobs))
print(rnd_data, "\n")

start_time = time.time()
sequence_list = all_permutations(init_job_list)
sol, M = brute_force(rnd_data, sequence_list)
elapsed_time = time.time() - start_time

print(f'Best sequence found by Brute Force {sol} with a makespan of {M}')
print("Elapsed time:", elapsed_time, "seconds")

[[22 17]
 [21 16]
 [ 6 22]
 [ 6 20]
 [15 19]
 [15 17]
 [21 20]] 

Best sequence found by Brute Force [2, 0, 3, 1, 4, 5, 6] with a makespan of 137.0
Elapsed time: 0.17924880981445312 seconds


# Branch and Bound

- Branch & Bound without pruning.
- Branch & Bound with pruning using `evaluate_sequence`.
- Branch & Bound with pruning using `lower_bound`.

In [9]:
# Define the Node structure of the seach tree that we will be using
class Node:
    def __init__(self, jobs, remaining_jobs, parent=None, lower_bound=1e100):
        self.jobs = jobs
        self.remaining_jobs = remaining_jobs
        self.parent = parent
        self.lower_bound = lower_bound
    def __str__(self):
        return f"Node(jobs={self.jobs}, remaining_jobs={self.remaining_jobs}, lower_bound={self.lower_bound})"

## Branch & Bound without pruning

In [10]:
def branch_and_bound_no_pruning(processing_times,initial_solution, initial_cost):
    jobs, machines = processing_times.shape
    # Initialize the nodes list to the `root_node`
    root_node = Node([], set(range(jobs)))
    best_solution = initial_solution.copy()
    best_solution_cost = initial_cost
    nodes = [root_node]
    # Initialize the `best_solution` to range(jobs) and initialize `best_solution_cost` its cost
    i = 1
    while nodes:
        node = nodes.pop()
        # Explore neighbours of the node `node`
        for job in node.remaining_jobs:
            child_jobs = node.jobs + [job]
            child_remaining_jobs = node.remaining_jobs - {job}
            # If the child node is a leaf node (i.e., all jobs have been assigned) then calculate its cost
            if not child_remaining_jobs:
                child_lower_bound = evaluate_sequence(child_jobs, processing_times)
                if child_lower_bound < best_solution_cost:
                    best_solution = child_jobs
                    best_solution_cost = child_lower_bound   
                    continue
            else:
                # If the child node is not a leaf then calculate its lower bound and compare it with current `best_solution_cost`
                child_lower_bound = evaluate_sequence(child_jobs, processing_times)
                child_node = Node(child_jobs, child_remaining_jobs, parent=node, lower_bound=child_lower_bound)
                nodes.append(child_node)
        i += 1
    return best_solution, best_solution_cost, i

In [27]:
# Parallelization for optimisation 
def sub_branch_and_bound_no_pruning(processing_times,initial_solution, initial_cost,root):
    jobs, machines = processing_times.shape
    # Creating threads (`jobs` thread) each one will explore a sub tree of the main tree.
    # There's no synchronization between them so they will all start with the same best_solution and best_solution_cost.
    # And by the end will take the best solution generated by each thread.
    # Not optimal but who cares :v at least it gets improved.
    # Initialize the nodes list to the `root_node`
    root_node = Node([root], set(range(jobs)) - {root}, [], processing_times[root, :].sum())
    best_solution = initial_solution.copy()
    best_solution_cost = initial_cost
    nodes = [root_node]
    # Initialize the `best_solution` to range(jobs) and initialize `best_solution_cost` its cost
    i = 1
    while nodes:
        node = nodes.pop()
        # Explore neighbours of the node `node`
        for job in node.remaining_jobs:
            child_jobs = node.jobs + [job]
            child_remaining_jobs = node.remaining_jobs - {job}
            # If the child node is a leaf node (i.e., all jobs have been assigned) then calculate its cost
            if not child_remaining_jobs:
                child_lower_bound = evaluate_sequence(child_jobs, processing_times)
                if child_lower_bound < best_solution_cost:
                    best_solution = child_jobs
                    best_solution_cost = child_lower_bound   
                    continue
            else:
                # If the child node is not a leaf then calculate its lower bound and compare it with current `best_solution_cost`
                child_lower_bound = evaluate_sequence(child_jobs, processing_times)
                child_node = Node(child_jobs, child_remaining_jobs, parent=node, lower_bound=child_lower_bound)
                nodes.append(child_node)
        i += 1
    return best_solution, best_solution_cost, i

## Branch & Bound with pruning using `evaluate_sequence`

In [11]:
def branch_and_bound_eval(processing_times, initial_solution, initial_cost):
    jobs, machines = processing_times.shape
    # Initialize the nodes list to the `root_node`
    root_node = Node([], set(range(jobs)))
    best_solution = initial_solution.copy()
    best_solution_cost = initial_cost
    print("M:", best_solution_cost)
    nodes = [root_node]
    # Initialize the `best_solution` to range(jobs) and initialize `best_solution_cost` its cost
    i = 1
    while nodes:
        node = nodes.pop()
        # Explore neighbours of the node `node`
        for job in node.remaining_jobs:
            child_jobs = node.jobs + [job]
            child_remaining_jobs = node.remaining_jobs - {job}
            child_lower_bound = evaluate_sequence(child_jobs, processing_times)
            # If the child node is a leaf node (i.e., all jobs have been assigned) then calculate its cost
            if not child_remaining_jobs:
                if child_lower_bound < best_solution_cost:
                    best_solution = child_jobs
                    best_solution_cost = child_lower_bound   
                    continue
            # If the child node is not a leaf then calculate its lower bound and compare it with current `best_solution_cost`
            if child_lower_bound < best_solution_cost:
                child_node = Node(child_jobs, child_remaining_jobs, parent=node, lower_bound=child_lower_bound)
                nodes.append(child_node)
        i += 1
    return best_solution, best_solution_cost, i

In [28]:
# Parallelization for optimisation 
def sub_branch_and_bound_eval(processing_times,initial_solution, initial_cost,root):
    jobs, machines = processing_times.shape
    # Creating threads (`jobs` thread) each one will explore a sub tree of the main tree.
    # There's no synchronization between them so they will all start with the same best_solution and best_solution_cost.
    # And by the end will take the best solution generated by each thread.
    # Not optimal but who cares :v at least it gets improved.
    # Initialize the nodes list to the `root_node`
    root_node = Node([root], set(range(jobs)) - {root}, [], processing_times[root, :].sum())
    best_solution = initial_solution.copy()
    best_solution_cost = initial_cost
    nodes = [root_node]
    # Initialize the `best_solution` to range(jobs) and initialize `best_solution_cost` its cost
    i = 1
    while nodes:
        node = nodes.pop()
        # Explore neighbours of the node `node`
        for job in node.remaining_jobs:
            child_jobs = node.jobs + [job]
            child_remaining_jobs = node.remaining_jobs - {job}
            child_lower_bound = evaluate_sequence(child_jobs, processing_times)
            # If the child node is a leaf node (i.e., all jobs have been assigned) then calculate its cost
            if not child_remaining_jobs:
                if child_lower_bound < best_solution_cost:
                    best_solution = child_jobs
                    best_solution_cost = child_lower_bound   
                    continue
            # If the child node is not a leaf then calculate its lower bound and compare it with current `best_solution_cost`
            if child_lower_bound < best_solution_cost:
                child_node = Node(child_jobs, child_remaining_jobs, parent=node, lower_bound=child_lower_bound)
                nodes.append(child_node)
        i += 1
    return best_solution, best_solution_cost, i

## Branch & Bound with pruning using `lower_bound`

In [12]:
'''
    This lowerbound was taken from the technical report
    Have no idea how It works, all I know is It's pretty slow
'''
def lower_bound(sequence, remaining_jobs, processing_times):
    nb_jobs, nb_machines = processing_times.shape
    total_completion = processing_times[sequence,:].sum(axis=0)
    lower_bound = 0
    for i in range(nb_machines):
        b = []
        a = []
        for j in sequence:
            b.append(processing_times[j][:i].sum())
            a.append(processing_times[j][i+1:].sum())
        l = min(a) + min(b) + total_completion[i]
        if (l > lower_bound):
            lower_bound = l
    return lower_bound

In [13]:
def branch_and_bound_lb(processing_times,initial_solution, initial_cost):
    jobs, machines = processing_times.shape
    # Initialize the nodes list to the `root_node`
    root_node = Node([], set(range(jobs)))
    best_solution = initial_solution.copy()
    best_solution_cost = initial_cost
    nodes = [root_node]
    # Initialize the `best_solution` to range(jobs) and initialize `best_solution_cost` its cost
    i = 1
    while nodes:
        node = nodes.pop()
        # Explore neighbours of the node `node`
        for job in node.remaining_jobs:
            child_jobs = node.jobs + [job]
            child_remaining_jobs = node.remaining_jobs - {job}
            # If the child node is a leaf node (i.e., all jobs have been assigned) then calculate its cost
            if not child_remaining_jobs:
                child_lower_bound = evaluate_sequence(child_jobs, processing_times)
                if child_lower_bound < best_solution_cost:
                    best_solution = child_jobs
                    best_solution_cost = child_lower_bound   
                    continue
            else:
                # If the child node is not a leaf then calculate its lower bound and compare it with current `best_solution_cost`
                child_lower_bound = lower_bound(child_jobs, child_remaining_jobs, processing_times)
                if child_lower_bound < best_solution_cost:
                    child_node = Node(child_jobs, child_remaining_jobs, parent=node, lower_bound=child_lower_bound)
                    nodes.append(child_node)
        i += 1
    return best_solution, best_solution_cost, i

# Tests

## Using Parallelized B&B Pure

In [23]:
import concurrent.futures

In [41]:
rnd_data = np.random.randint(size=(5,10), low=5, high=23)
print(rnd_data, "\n")

[[ 8 10 18 10  6 21 12 12  6  8]
 [10 20 15 12 22 16 11 16 18  6]
 [ 8 16 22  9 13 19  6 21 19 11]
 [12 12 21  9 17 11 15 17 17 21]
 [11 13 21  9  9  9 22 10 12  5]] 



In [47]:
initial_solution = [i for i in range(5)]
initial_cost = evaluate_sequence(initial_solution, rnd_data)

# Code starts here
start_time = time.time()
results = []
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Submit the function to the executor five times
    futures = [executor.submit(sub_branch_and_bound_no_pruning, rnd_data, initial_solution, initial_cost, i) for i in range(5)]
    
    # Collect the results from the futures
    results = [future.result() for future in futures]

elapsed_time = time.time() - start_time
print(f'Results are: {results}')
min_tuple = min(results, key=lambda x: x[1])
print(f'\nBest sequence is {min_tuple[0]} with a makespan of {min_tuple[1]} ')
print("Elapsed time:", elapsed_time, "seconds")

Results are: [([0, 1, 3, 2, 4], 204.0, 42), ([1, 3, 2, 4, 0], 205.0, 42), ([2, 3, 1, 4, 0], 203.0, 42), ([3, 2, 1, 4, 0], 203.0, 42), ([4, 3, 2, 1, 0], 212.0, 42)]

Best sequence is [2, 3, 1, 4, 0] with a makespan of 203.0 
Elapsed time: 0.05596518516540527 seconds


In [48]:
initial_solution = [i for i in range(5)]
initial_cost = evaluate_sequence(initial_solution, rnd_data)

start_time = time.time()
best_solution, best_cost, i = branch_and_bound_no_pruning(rnd_data, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'\nBest sequence is {best_solution} with a makespan of {best_cost}')
print("Elapsed time:", elapsed_time, "seconds")


Best sequence is [3, 2, 1, 4, 0] with a makespan of 203.0
Elapsed time: 0.04200243949890137 seconds


## Common Instance

In [22]:
instance_common = np.array([
    [71, 79, 85, 82, 83], 
    [84, 71, 66, 68, 81],
    [78, 81, 75, 72, 87],
    [78, 75, 66, 72, 88],
    [72, 88, 83, 85, 88],
    [86, 88, 79, 82, 78],
    [75, 66, 86, 78, 78],
    [80, 79, 66, 83, 78],
    [73, 73, 67, 77, 71],
    [80, 77, 83, 78, 67],
])

### B&B with `evaluate_sequence`

#### Random initial solution

In [56]:
initial_solution = [i for i in range(10)]
initial_cost = evaluate_sequence(initial_solution, instance_common)

start_time = time.time()
best_solution, best_cost, i = branch_and_bound_eval(instance_common, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'\nBest sequence is {best_solution} with a makespan of {best_cost}')
print("Elapsed time:", elapsed_time, "seconds")

M: 1116.0

Best sequence is [3, 2, 7, 6, 8, 0, 1, 4, 5, 9] with a makespan of 1102.0
Elapsed time: 1016.035453081131 seconds


#### Solution initialized with NEH

In [54]:
initial_solution, initial_cost = neh_algorithm(instance_common)

start_time = time.time()
best_solution, best_cost, i = branch_and_bound_eval(instance_common, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'\nBest sequence is {best_solution} with a makespan of {best_cost}')
print("Elapsed time:", elapsed_time, "seconds")

M: 1105.0

Best sequence is [3, 2, 7, 6, 8, 0, 1, 4, 5, 9] with a makespan of 1102.0
Elapsed time: 1040.4345321655273 seconds


### B&B pure

In [73]:
initial_solution = [i for i in range(10)]
initial_cost = evaluate_sequence(initial_solution, instance_common)

start_time = time.time()
best_solution, best_cost, i = branch_and_bound_no_pruning(instance_common, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'\nBest sequence is {best_solution} with a makespan of {best_cost}')
print("Elapsed time:", elapsed_time, "seconds")


Best sequence is [3, 2, 7, 6, 8, 0, 1, 4, 5, 9] with a makespan of 1102.0
Elapsed time: 1248.8050439357758 seconds


## Taillard Instances

In [75]:
# Open the file that contains the instances
file = open("Benchmarks/tai20_5.txt", "r")

# Read the file line by line to retrieve the instances
n = 0
instances = [[]]
line = file.readline()

while line:
    if line != '\n':
        line = line.strip(' ')
        line = line[:-1]
        line = line.split()
        line = [int(num) for num in line]
        instances[n].append(line)
    else:
        instances.append([])
        n += 1
    line = file.readline()
    
print(f'Number of instances in this file {len(instances)}.')

for instance in instances:
    print("\n")
    print(instance)
    

Number of instances in this file 10.


[[54, 83, 15, 71, 77, 36, 53, 38, 27, 87, 76, 91, 14, 29, 12, 77, 32, 87, 68, 94], [79, 3, 11, 99, 56, 70, 99, 60, 5, 56, 3, 61, 73, 75, 47, 14, 21, 86, 5, 77], [16, 89, 49, 15, 89, 45, 60, 23, 57, 64, 7, 1, 63, 41, 63, 47, 26, 75, 77, 40], [66, 58, 31, 68, 78, 91, 13, 59, 49, 85, 85, 9, 39, 41, 56, 40, 54, 77, 51, 31], [58, 56, 20, 85, 53, 35, 53, 41, 69, 13, 86, 72, 8, 49, 47, 87, 58, 18, 68, 28]]


[[26, 38, 27, 88, 95, 55, 54, 63, 23, 45, 86, 43, 43, 40, 37, 54, 35, 59, 43, 50], [59, 62, 44, 10, 23, 64, 47, 68, 54, 9, 30, 31, 92, 7, 14, 95, 76, 82, 91, 37], [78, 90, 64, 49, 47, 20, 61, 93, 36, 47, 70, 54, 87, 13, 40, 34, 55, 13, 11, 5], [88, 54, 47, 83, 84, 9, 30, 11, 92, 63, 62, 75, 48, 23, 85, 23, 4, 31, 13, 98], [69, 30, 61, 35, 53, 98, 94, 33, 77, 31, 54, 71, 78, 9, 79, 51, 76, 56, 80, 72]]


[[77, 94, 9, 57, 29, 79, 55, 73, 65, 86, 25, 39, 76, 24, 38, 5, 91, 29, 22, 27], [39, 31, 46, 18, 93, 58, 85, 58, 97, 10, 79, 93, 2, 87, 17, 18, 10, 